In [1]:
import pandas as pd

In [3]:
import sys
import os

# Add project root to Python path
sys.path.append(os.path.abspath(".."))


In [5]:
# Load saved player data
df = pd.read_csv("../data/sleeper_players_sample.csv")
df.head()


,player_id,full_name,position,team,age,status,injury_status
0,6462,Ellis Richardson,TE,NaN,26.0,Active,NaN
1,11255,Nick Amoah,OL,NaN,NaN,Active,NaN
2,8842,Malkelm Morrison,CB,NaN,NaN,Injured Reserve,NaN
3,7926,Carl Tucker,TE,NaN,24.0,Active,NaN
4,1875,C.J. Mosley,LB,NaN,32.0,Active,Questionable


In [7]:
import os

# Show where the notebook is running from
print("Notebook current working directory:", os.getcwd())

# Show files in the current dir
print("\nFiles in current folder:")
print(os.listdir())

# Show files one level up
print("\nFiles in parent folder:")
print(os.listdir(".."))

# Show what's in the /data folder (relative to notebook)
print("\nFiles in ../data:")
print(os.listdir("../data"))


Notebook current working directory: /Users/ryanhofmann/ffanalyticspro/notebooks

Files in current folder:
['Start_Sit_Logic_Engine.ipynb', 'sleeper_player_data.ipynb', '.ipynb_checkpoints']

Files in parent folder:
['ui', 'enhanced_start_sit_demo.py', 'tests', 'models', 'README.md', 'api', '.ipynb_checkpoints', '.git', 'data', 'notebooks', 'test_scoring_formats.py']

Files in ../data:
['sleeper_players_sample.csv', 'sleeper_player_data.ipynb', '.ipynb_checkpoints']


In [9]:
## Mock Fantasy Roster
# Filter down to a mock roster (you can update this later)
# Show what teams and positions are actually in your dataset
print("Available teams:", df['team'].dropna().unique())
print("Available positions:", df['position'].dropna().unique())



Available teams: ['TEN' 'TB' 'ARI' 'DAL' 'SEA' 'BAL' 'JAX' 'CIN' 'IND' 'SF' 'NYG' 'PHI'
 'NO' 'CAR' 'LAR' 'CLE' 'ATL' 'CHI' 'OAK' 'LAC']
Available positions: ['TE' 'OL' 'CB' 'LB' 'FS' 'RB' 'QB' 'K' 'DE' 'WR' 'DT' 'T' 'DL' 'DB' 'OT'
 'G' 'C' 'SS' 'LS']


In [11]:
# Build mock fantasy roster using any available active players
mock_roster = df[
    (df['position'].isin(['QB', 'RB', 'WR', 'TE'])) &
    (df['status'] == 'Active')
]

# Add fake projections again
import numpy as np
np.random.seed(42)
mock_roster = mock_roster.copy()
mock_roster['projected_points'] = np.random.uniform(5, 30, len(mock_roster))

# Reset index
mock_roster = mock_roster.reset_index(drop=True)

# Show the new mock roster
print("✅ Mock roster shape:", mock_roster.shape)
mock_roster[['full_name', 'position', 'team', 'projected_points']].head()


✅ Mock roster shape: (25, 8)


,full_name,position,team,projected_points
0,Ellis Richardson,TE,NaN,14.363503
1,Carl Tucker,TE,NaN,28.767858
2,Le'Veon Bell,RB,TB,23.299849
3,Matt Seybert,TE,NaN,19.966462
4,Kinkead Dent,QB,NaN,8.900466


In [13]:
STARTING_SLOTS = {
    'QB': 1,
    'RB': 2,
    'WR': 2,
    'TE': 1,
}


In [15]:
recommendations = []

for position in STARTING_SLOTS.keys():
    pos_players = mock_roster[mock_roster['position'] == position]
    num_to_start = STARTING_SLOTS[position]
    
    pos_players = pos_players.sort_values(by='projected_points', ascending=False)
    
    for i, (_, row) in enumerate(pos_players.iterrows()):
        recommendations.append({
            "full_name": row.get('full_name', ''),
            "position": row.get('position', ''),
            "team": row.get('team', ''),
            "projected_points": row.get('projected_points', 0),
            "recommendation": "START" if i < num_to_start else "SIT"
        })

# Convert to DataFrame
final_df = pd.DataFrame(recommendations)

# Double-check columns
print(final_df.columns)

# Now sort safely
final_df = final_df.sort_values(by=['position', 'recommendation'], ascending=[True, False])
final_df.head()


Index(['full_name', 'position', 'team', 'projected_points', 'recommendation'], dtype='object')


,full_name,position,team,projected_points,recommendation
0,Brett Gabbert,QB,NaN,26.654404,START
1,Kinkead Dent,QB,NaN,8.900466,SIT
2,Le'Veon Bell,RB,TB,23.299849,START
3,Bucky Irving,RB,TB,15.798625,START
4,Kendall Milton,RB,CIN,12.280729,SIT


In [17]:
!pip install selenium webdriver-manager


In [18]:
!pip install selenium

In [21]:
import importlib
import api.fantasypros_scraper as scraper

importlib.reload(scraper)
from api.fantasypros_scraper import scrape_rankings

df_rb = scrape_rankings('RB')
df_rb.head()


🌐 Fetching: https://www.fantasypros.com/nfl/rankings/rb.php
✅ Successfully scraped 127 RB players for Standard scoring


,name,team,opponent,proj_fpts,rank,start_sit,position,scoring_format
0,Saquon Barkley,PHI,vs. DAL,17.8,1,A+,RB,Standard
1,Bijan Robinson,ATL,vs. TB,15.8,2,A+,RB,Standard
2,Derrick Henry,BAL,at BUF,15.4,3,A,RB,Standard
3,Christian McCaffrey,SF,at SEA,14.8,4,A,RB,Standard
4,Jahmyr Gibbs,DET,at GB,14.7,5,A,RB,Standard


In [23]:
# Test the enhanced scraper with different scoring formats
print("🚀 Testing Enhanced FantasyPros Scraper with Scoring Formats")
print("=" * 60)

# Test different scoring formats
scoring_formats = ['Standard', 'PPR', 'Half-PPR']
position = 'RB'

results = {}

for scoring_format in scoring_formats:
    print(f"\n📊 Scraping {position} rankings for {scoring_format} scoring...")
    try:
        df = scrape_rankings(position, scoring_format)
        if not df.empty:
            results[scoring_format] = df
            print(f"✅ Successfully scraped {len(df)} players")
            print(f"Top 3 players:")
            print(df[['name', 'team', 'proj_fpts', 'start_sit']].head(3).to_string(index=False))
        else:
            print(f"❌ No data retrieved for {scoring_format}")
    except Exception as e:
        print(f"❌ Error scraping {scoring_format}: {e}")


🚀 Testing Enhanced FantasyPros Scraper with Scoring Formats

📊 Scraping RB rankings for Standard scoring...
🌐 Fetching: https://www.fantasypros.com/nfl/rankings/rb.php
✅ Successfully scraped 127 RB players for Standard scoring
✅ Successfully scraped 127 players
Top 3 players:
          name team  proj_fpts start_sit
Saquon Barkley  PHI       17.8        A+
Bijan Robinson  ATL       15.8        A+
 Derrick Henry  BAL       15.4         A

📊 Scraping RB rankings for PPR scoring...
🌐 Fetching: https://www.fantasypros.com/nfl/rankings/ppr-rb.php
✅ Successfully scraped 127 RB players for PPR scoring
✅ Successfully scraped 127 players
Top 3 players:
               name team  proj_fpts start_sit
     Bijan Robinson  ATL       20.2        A+
     Saquon Barkley  PHI       18.8        A+
Christian McCaffrey   SF       18.1        A+

📊 Scraping RB rankings for Half-PPR scoring...
🌐 Fetching: https://www.fantasypros.com/nfl/rankings/half-point-ppr-rb.php
✅ Successfully scraped 171 RB players for

In [ ]:
# Improved comparison across scoring formats (NO MORE NaN VALUES!)
if len(results) > 1:
    print(f"\n📈 Comparing ALL RB Players Across Scoring Formats:")
    print("=" * 60)
    
    # Get all unique players across formats
    all_players = set()
    for df in results.values():
        all_players.update(df['name'].tolist())
    
    print(f"📊 Found {len(all_players)} unique players across all formats")
    
    # Create comprehensive comparison data
    comparison_data = []
    for player_name in sorted(all_players):
        player_data = {'name': player_name}
        
        # Get data for this player from each format
        for scoring_format, df in results.items():
            player_row = df[df['name'] == player_name]
            if not player_row.empty:
                player_data[f'{scoring_format}_proj'] = player_row.iloc[0]['proj_fpts']
                player_data[f'{scoring_format}_rank'] = player_row.iloc[0]['rank']
                player_data[f'{scoring_format}_team'] = player_row.iloc[0]['team']
            else:
                player_data[f'{scoring_format}_proj'] = None
                player_data[f'{scoring_format}_rank'] = None
                player_data[f'{scoring_format}_team'] = None
        
        comparison_data.append(player_data)
    
    comparison_df = pd.DataFrame(comparison_data)
    
    # Clean up the data - use the most common team for each player
    for idx, row in comparison_df.iterrows():
        teams = [row['Standard_team'], row['PPR_team'], row['Half-PPR_team']]
        teams = [t for t in teams if t is not None]
        if teams:
            most_common_team = max(set(teams), key=teams.count)
            comparison_df.at[idx, 'team'] = most_common_team
        else:
            comparison_df.at[idx, 'team'] = 'Unknown'
    
    # Create a cleaner comparison table
    clean_comparison = comparison_df[['name', 'team']].copy()
    for scoring_format in ['Standard', 'PPR', 'Half-PPR']:
        clean_comparison[f'{scoring_format}_proj'] = comparison_df[f'{scoring_format}_proj']
        clean_comparison[f'{scoring_format}_rank'] = comparison_df[f'{scoring_format}_rank']
    
    # Remove players with no projections in any format
    clean_comparison = clean_comparison.dropna(subset=['Standard_proj', 'PPR_proj', 'Half-PPR_proj'], how='all')
    
    # Sort by average projection across formats
    proj_cols = ['Standard_proj', 'PPR_proj', 'Half-PPR_proj']
    clean_comparison['avg_projection'] = clean_comparison[proj_cols].mean(axis=1, skipna=True)
    clean_comparison = clean_comparison.sort_values('avg_projection', ascending=False)
    
    # Show top 10 players
    print(f"\n🏆 Top 10 Players Comparison:")
    print("=" * 80)
    
    top_players = clean_comparison.head(10)
    for idx, row in top_players.iterrows():
        print(f"\n{row['name']} ({row['team']})")
        print("-" * 40)
        
        for scoring_format in ['Standard', 'PPR', 'Half-PPR']:
            proj = row[f'{scoring_format}_proj']
            rank = row[f'{scoring_format}_rank']
            if pd.notna(proj):
                print(f"  {scoring_format:>10}: {proj:>5.1f} pts (Rank #{rank})")
            else:
                print(f"  {scoring_format:>10}: Not ranked")
    
    # PPR Advantage Analysis
    print(f"\n📊 PPR Advantage Analysis (Top 10):")
    print("=" * 60)
    
    # Calculate PPR advantage
    clean_comparison['ppr_advantage'] = clean_comparison['PPR_proj'] - clean_comparison['Standard_proj']
    
    # Filter to players with both Standard and PPR projections
    valid_players = clean_comparison.dropna(subset=['Standard_proj', 'PPR_proj'])
    
    # Sort by PPR advantage
    ppr_advantage = valid_players.sort_values('ppr_advantage', ascending=False).head(10)
    
    print(f"{'Player':<20} {'Team':<4} {'Standard':<8} {'PPR':<6} {'Advantage':<10}")
    print("-" * 60)
    
    for idx, row in ppr_advantage.iterrows():
        print(f"{row['name']:<20} {row['team']:<4} {row['Standard_proj']:<8.1f} {row['PPR_proj']:<6.1f} {row['ppr_advantage']:<10.1f}")



📈 Comparing Top 5 RB Projections Across Scoring Formats:
               name team  Half-PPR  PPR  Standard
     Bijan Robinson  ATL      17.1 20.2      15.8
        Chase Brown  CIN       NaN 17.4       NaN
Christian McCaffrey   SF      16.5 18.1      14.8
      Derrick Henry  BAL      16.1  NaN      15.4
       Jahmyr Gibbs  DET      16.3 17.9      14.7
     Saquon Barkley  PHI      19.0 18.8      17.8

📊 PPR vs Standard Difference (Top 5):
               name team  Standard  PPR  PPR_vs_Standard
     Bijan Robinson  ATL      15.8 20.2              4.4
        Chase Brown  CIN       NaN 17.4              NaN
Christian McCaffrey   SF      14.8 18.1              3.3
      Derrick Henry  BAL      15.4  NaN              NaN
       Jahmyr Gibbs  DET      14.7 17.9              3.2


In [ ]:
# Enhanced Start/Sit Logic using FantasyPros data
def create_start_sit_recommendations(scoring_format='Standard'):
    """
    Create start/sit recommendations using FantasyPros data.
    """
    print(f"🎯 Creating Start/Sit Recommendations for {scoring_format} Scoring")
    print("=" * 60)
    
    # Define starting lineup slots
    STARTING_SLOTS = {
        'QB': 1,
        'RB': 2,
        'WR': 2,
        'TE': 1,
    }
    
    all_recommendations = []
    
    for position in STARTING_SLOTS.keys():
        print(f"\n📊 Processing {position} recommendations...")
        
        # Scrape data for this position
        df = scrape_rankings(position, scoring_format)
        
        if df.empty:
            print(f"⚠️ No data available for {position}")
            continue
            
        # Sort by projected points
        df_sorted = df.sort_values('proj_fpts', ascending=False)
        
        num_to_start = STARTING_SLOTS[position]
        
        for i, (_, row) in enumerate(df_sorted.iterrows()):
            recommendation = "START" if i < num_to_start else "SIT"
            
            all_recommendations.append({
                'name': row['name'],
                'position': position,
                'team': row['team'],
                'opponent': row['opponent'],
                'proj_fpts': row['proj_fpts'],
                'rank': row['rank'],
                'start_sit_grade': row['start_sit'],
                'recommendation': recommendation,
                'scoring_format': scoring_format
            })
    
    return pd.DataFrame(all_recommendations)

# Test with PPR scoring
ppr_recommendations = create_start_sit_recommendations('PPR')
print(f"\n✅ Generated {len(ppr_recommendations)} recommendations")

# Show top recommendations by position
for position in ['QB', 'RB', 'WR', 'TE']:
    pos_recs = ppr_recommendations[ppr_recommendations['position'] == position]
    starters = pos_recs[pos_recs['recommendation'] == 'START']
    print(f"\n{position} Starters:")
    print(starters[['name', 'team', 'opponent', 'proj_fpts', 'start_sit_grade']].to_string(index=False))


🎯 Creating Start/Sit Recommendations for PPR Scoring

📊 Processing QB recommendations...
🌐 Fetching: https://www.fantasypros.com/nfl/rankings/ppr-qb.php
